# Vault and Kubernetes Setup

Vault Agent takes responsibility for these tasks and enables your applications to
remain unaware of Vault. However, this introduces a new requirement that
deployments install and configure Vault Agent alongside the application as a
sidecar.

The Vault Helm chart enables you to run Vault and the Vault Agent Injector
service. This injector service leverages the Kubernetes mutating admission
webhook to intercept pods that define specific annotations and inject a Vault
Agent container to manage these secrets. This is beneficial because:

- Applications remain Vault unaware as the secrets are stored on the file-system
  in their container.
- Existing deployments require no change; as annotations can be patched.
- Access to secrets can be enforced via Kubernetes service accounts and
  namespaces

In this tutorial, you setup Vault and this injector service with the Vault Helm
chart. Then you will deploy several applications to demonstrate how this new injector
service retrieves and writes these secrets for the applications to use.

## Prerequisites

This tutorial requires:
* the [Kubernetes command-line interface
(CLI)](https://kubernetes.io/docs/tasks/tools/install-kubectl/) 
* the [Helm
CLI](https://helm.sh/docs/helm/) installed
* [kind](https://kind.sigs.k8s.io/) or [Minikube](https://minikube.sigs.k8s.io)
* and additional configuration to bring it all together.

This tutorial was last tested 23 Apr 2021 on a macOS 11.2.3 using this
configuration.

* Docker version: 20.10.6
* kind version: 0.11.1
* Minikube version: - ?
* helm version: 3.6.3

In [ ]:
echo "#==> docker version"
docker version | grep -i version
echo "#==> kind version"
kind version
echo "#==> minikube version"
minikube version
echo "#==> helm version"
helm version

### Install software

#### Instal minikube kubernetes

Follow the directions to [install
Minikube](https://kubernetes.io/docs/tasks/tools/install-minikube/), including
VirtualBox or similar.

#### Install kind kubernetes

In [ ]:
brew install kind

#### Install kubectl CLI and helm CLI

Install [kubectl CLI](https://kubernetes.io/docs/tasks/tools/install-kubectl/)
and [helm CLI](https://github.com/helm/helm#install).

**Homebrew on OS X**

Install `kubectl`  and `helm` with [Homebrew](https://brew.sh).

In [ ]:
brew install kubernetes-cli
brew install helm

**Chocolatey on Windows**

Install `kubectl` and `helm` with [Chocolatey](https://chocolatey.org/).

```shell
$ choco install kubernetes-cli
$ choco install kubernetes-helm
```

(optional) Install Lens IDE

In [ ]:
brew install --cask lens

### Clone Git Repo

Next, retrieve the web application and additional configuration by cloning the
[hashicorp/vault-guides](https://github.com/hashicorp/vault-guides) repository
from GitHub.

In [ ]:
git clone https://github.com/hashicorp/vault-guides.git
# GIT_DIR=vault-agent-sidecar

This repository contains supporting content for all of the Vault learn guides.
The content specific to this tutorial can be found in a sub-directory.

Go into the
`vault-guides/operations/provision-vault/kubernetes/minikube/vault-agent-sidecar`
directory.

In [ ]:
pushd vault-guides/operations/provision-vault/kubernetes/minikube/vault-agent-sidecar

> **Working directory:** This tutorial assumes that the remainder of commands
are executed in this directory.

## Create a kind Kubernetes cluster

Create a kind Kubernetes cluster called `vault-learn`. This takes approximately 2 minutes.

In [ ]:
kind create cluster --name vault-learn

Sample Output

```shell
Creating cluster "vault-learn" ...
 ✓ Ensuring node image (kindest/node:v1.21.1) 🖼
 ✓ Preparing nodes 📦
 ✓ Writing configuration 📜
 ✓ Starting control-plane 🕹️
 ✓ Installing CNI 🔌
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-vault-learn"
You can now use your cluster with:

kubectl cluster-info --context kind-vault-learn
```

### Verify kind cluster

Confirm that `kind` created a container for you.

In [ ]:
docker ps -a | grep kind

Verify that your kubernetes cluster exists by listing your `kind` clusters.

In [ ]:
kind get clusters

Sample Output
```shell
vault-learn
```

Then, point `kubectl` to interact with this cluster.

In [ ]:
kubectl cluster-info --context kind-vault-learn

Sample Output
```shell
Kubernetes master is running at https://127.0.0.1:32769
KubeDNS is running at https://127.0.0.1:32769/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
```

### Optional - Start Minikube

[Minikube](https://minikube.sigs.k8s.io/) is a CLI tool that provisions and
manages the lifecycle of single-node [Kubernetes
clusters](https://kubernetes.io/docs/concepts/architecture/) locally
inside Virtual Machines (VM) on your system.

Start a Kubernetes cluster.

```shell-session
$ minikube start
😄  minikube v1.19.0 on Darwin 11.2.3
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🔄  Restarting existing docker container for "minikube" ...
🐳  Preparing Kubernetes v1.20.2 on Docker 20.10.5 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
```

The initialization process takes several minutes as it retrieves any necessary
dependencies and executes various container images.

Verify the status of the Minikube cluster.

```shell-session
$ minikube status
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured
```

> **Additional waiting:** Even if this last command completed successfully, you
may have to wait for Minikube to be available. If an error is displayed, try
again after a few minutes.

The host, kubelet, and apiserver report that they are running. The `kubectl`, a
command line interface (CLI) for running commands against Kubernetes cluster, is
also configured to communicate with this recently started cluster.

Minikube provides a visual representation of the status in a web-based
dashboard. This interface displays the cluster activity in a visual interface
that can assist in delving into the issues affecting it.

In **another terminal**, launch the minikube dashboard.

```shell-session
$ minikube dashboard
```

The operating system's default browser opens and displays the dashboard.

![Minikube Dashboard](/img/vault-k8s/minikube-dashboard.png)

## Install the Vault Helm chart

The recommended way to run Vault on Kubernetes is via the [Helm
chart](https://www.vaultproject.io/docs/platform/k8s/helm.html).
[Helm](https://helm.sh/docs/helm/) is a package manager that installs and
configures all the necessary components to run Vault in several different
modes. A Helm chart includes
[templates](https://helm.sh/docs/chart_template_guide)
that enable conditional and parameterized execution. These parameters can be set
through command-line arguments or defined in YAML.

Create Kubernetes namespace for Vault called `vault`.

In [ ]:
kubectl create ns vault

In [ ]:
kubectl get ns
printf "\n# View your new k8s objects\n"
kubectl -n vault get all

Add the HashiCorp Helm repository.

In [ ]:
helm repo add hashicorp https://helm.releases.hashicorp.com

```shell
"hashicorp" has been added to your repositories
```

Update all the repositories to ensure `helm` is aware of the latest versions.

In [ ]:
helm repo update

```shell
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "hashicorp" chart repository
Update Complete. ⎈Happy Helming!⎈
```

Run `helm install` as a dry-run.

In [ ]:
helm install vault hashicorp/vault --namespace=default \
  --set "server.dev.enabled=true" --dry-run

Install the latest version of the Vault server running in development mode.

In [ ]:
helm search repo hashicorp/vault --versions

### Override default settings.

In [ ]:
helm install vault hashicorp/vault \
  --namespace vault \
  --set "server.ha.enabled=true" \
  --set "server.ha.replicas=5" \
  --dry-run

Alternatively, specify the desired configuration in a file `override-values.yml`

In [ ]:
cat > ./override-values.yml << EOF
server:
  ha:
    enabled: true
    replicas: 5
EOF

In [ ]:
helm install vault hashicorp/vault \
  --namespace vault \
  -f override-values.yml # \
  --dry-run

In [ ]:
helm install vault hashicorp/vault \
  --namespace=vault \
  --set "server.dev.enabled=true" 
  # \
  # --set "injector.enabled=false" \
  # --set "csi.enabled=true"

```shell-session
NAME: vault
##...
```

The Vault pod and Vault Agent Injector pod are deployed in the `default`
namespace.

Display all the pods in the `default` namespace.

In [ ]:
kubectl get pods -n vault

```shell
NAME                                    READY   STATUS    RESTARTS   AGE
vault-0                                 1/1     Running   0          80s
vault-agent-injector-5945fb98b5-tpglz   1/1     Running   0          80s
```

The `vault-0` pod runs a Vault server in development mode. The
`vault-agent-injector` pod performs the injection based on the annotations
present or patched on a deployment.

> **Development mode:** Running a Vault server in development is automatically
initialized and unsealed. This is ideal in a learning environment but NOT
recommended for a production environment.

Wait until the `vault-0` pod and `vault-agent-injector` pod are `Running` and
`Ready` (`1/1`).

## Configure Vault

Create a bash alias for the `vault_0` command.

In [ ]:
# set mysql alias
alias vault_0="kubectl exec -it vault-0 -n vault -- vault"

## Clean Up Resources

In [ ]:
helm uninstall vault -n vault 

### Delete `kind` cluster

Finally, delete the `kind` cluster.

In [ ]:
kind delete cluster --name vault-learn

Sample Output
```shell
Deleting cluster "terraform-learn" ...
```

### Delete artifacts

**CAUTION**: Make sure you no longer need these before deleting.

In [ ]:
# Go back to original folder
popd
# Confirm you are in correct directory and can see the dir you're going to delete.
pwd; echo; ls -l

In [ ]:
rm -rf vault-guides

## Resources

* [Learn - Injecting Secrets into Kubernetes Pods via Vault Agent Containers](https://learn.hashicorp.com/tutorials/vault/kubernetes-sidecar#define-a-kubernetes-service-account)
* [Doc - K8s integration](https://www.vaultproject.io/docs/platform/k8s/)
* [Doc - K8s Helm Chart](https://www.vaultproject.io/docs/platform/k8s/helm/configuration)
* [Vault Helm project source code](https://github.com/hashicorp/vault-helm)
* [Doc - Agent Sidecar
Injector](https://www.vaultproject.io/docs/platform/k8s/injector/index.html)

## Troubleshooting

### Connect to Vault UI

```shell
kubectl port-forward svc/vault 8200:8200
```

In [ ]:
echo $VAULT_ADDR
export VAULT_ADDR=http://localhost:8200
export VAULT_TOKEN=root

Create a read-only policy, `myapp-kv-ro` in Vault.

In [ ]:
vault policy write myapp-kv-ro - <<EOF
path "secret/data/myapp/*" {
    capabilities = ["read", "list"]
}
EOF

Create some test data at the `secret/myapp` path.

In [ ]:
vault kv put secret/myapp/config \
username='appuser' \
password='suP3rsec(et!' \
ttl='30s'

In [ ]:
vault kv get secret/myapp/config

#### Set environment variables required for Vault configuration.

https://docs.armory.io/docs/armory-admin/secrets/vault-k8s-configuration/
https://learn.hashicorp.com/tutorials/vault/agent-kubernetes?in=vault/kubernetes

In [ ]:
# Set VAULT_SA_NAME to the service account you created earlier
export VAULT_SA_NAME=$(kubectl -n default get sa vault-auth -o jsonpath="{.secrets[*]['name']}")
echo $VAULT_SA_NAME

In [ ]:
# Set SA_JWT_TOKEN value to the service account JWT used to access the TokenReview API
export SA_JWT_TOKEN=$(kubectl get secret $VAULT_SA_NAME \
  --output 'go-template={{ .data.token }}' | base64 --decode)
echo $SA_JWT_TOKEN

In [ ]:
# Set SA_CA_CRT to the PEM encoded CA cert used to talk to Kubernetes API
export SA_CA_CRT=$(kubectl config view --raw --minify --flatten \
  --output 'jsonpath={.clusters[].cluster.certificate-authority-data}' \
  | base64 --decode)
echo $SA_CA_CRT

In [ ]:
#Set the K8S_HOST environment variable value to minikube IP address.
export K8S_HOST=$(kubectl config view --raw --minify --flatten \
    --output 'jsonpath={.clusters[].cluster.server}')
echo $K8S_HOST

In [ ]:
#Tell Vault how to communicate with the Kubernetes (Minikube) cluster.
vault write auth/kubernetes/config \
    issuer="https://kubernetes.default.svc.cluster.local" \
    token_reviewer_jwt="$SA_JWT_TOKEN" \
    kubernetes_host="$K8S_HOST" \
    kubernetes_ca_cert="$SA_CA_CRT"

In [ ]:
vault read auth/kubernetes/config

In [ ]:
# Create a role named, example, that maps the Kubernetes Service Account to Vault policies and default token TTL.
vault write auth/kubernetes/role/example \
        bound_service_account_names=vault-auth \
        bound_service_account_namespaces=default \
        policies=myapp-kv-ro \
        ttl=24h

In [ ]:
export EXTERNAL_VAULT_ADDR=vault

In [ ]:
tee devwebapp.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: devwebapp
  labels:
    app: devwebapp
spec:
  serviceAccountName: internal-app
  containers:
    - name: devwebapp
      image: burtlo/devwebapp-ruby:k8s
      env:
        - name: VAULT_ADDR
          value: "http://$EXTERNAL_VAULT_ADDR:8200"
        - name: VAULT_TOKEN
          value: root
EOF

In [ ]:
kubectl apply --filename devwebapp.yaml --namespace default

In [ ]:
kubectl get pods

In [ ]:
# kubectl exec --stdin=true --tty=true devwebapp -- /bin/sh


In [ ]:
cat > vault-auth-service-account.yml <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: role-tokenreview-binding
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: system:auth-delegator
subjects:
- kind: ServiceAccount
  name: vault-auth
  namespace: default
EOF

In [ ]:
# Create a service account, 'vault-auth'
kubectl -n default create serviceaccount vault-auth

In [ ]:
# Update the 'vault-auth' service account
kubectl -n default apply --filename vault-auth-service-account.yml

In [ ]:
cat patch-inject-secrets.yaml

Create a Kubernetes authentication role named `internal-app`.

```shell-session
$ vault write auth/kubernetes/role/internal-app \
    bound_service_account_names=internal-app \
    bound_service_account_namespaces=default \
    policies=internal-app \
    ttl=24h
Success! Data written to: auth/kubernetes/role/internal-app
```

In [ ]:
k exec -it vault-0 -- sh -c "vault write auth/kubernetes/role/internal-app \
    bound_service_account_names=internal-app \
    bound_service_account_namespaces=default \
    policies=internal-app \
    ttl=24h"

In [ ]:
# VERIFY
kubectl exec -it vault-0 -- sh -c "vault read auth/kubernetes/role/internal-app"

Check vault logs. You can find your unseal keys and root token here.

In [ ]:
kubectl logs vault-0 -n vault

(optional) Helm uninstall vault

In [ ]:
helm uninstall vault -n vault